In [1]:
import sys; sys.path.append("vplatoon")

In [2]:
%load_ext autoreload

In [3]:
autoreload 2

# Project Description 

## Analyzing Platooning Technologies and their benefits for transportation

The main objective of this project is to understand a traffic phenomena denominated "platoon" and design proper control strategies for a platoon of vehicles in order to optimize the traffic flow and reduce the fuel consumption. 

<img src="assets/img/truck-platooning.jpg" alt="drawing" width="600"/>

In order to do that we are going to make use of the package `vplatoon` that includes some basic models to deploy platooning systems including the implemenation of [`ACC/CACC`](https://en.wikipedia.org/wiki/Adaptive_cruise_control) systems. For more information about the source code of this package please go [here](https://github.com/aladinoster/vplatoon). Over there you will find a folder called `vplatoon` containing all the models in the submodules `carfollow`, `vehicles`, `demand`, `messages` etc. Some of these will be explored along the module and may help you to progress faster in the project.   


In [4]:
# import sys; sys.path.append("connectv2x") # Uncomment in case pip does not work 

## Contents and structure 

* Exploration and modeling of vehicle dynamics and 
* Basic control systems 
* Implementation of a platoon via control systems 
* Compute emissions from trajectories 
* Understanding side effects of platoon (Stability) 

Let's begin by importing some plotting libraries that will help us with data visualization

In [5]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Understanding the basics 

For this application it is considered the simulation of [microscopic traffic models](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) where *longitudinal position* follow a specified behavior defined as a function of  two main components, the *headway space* and the *speed differential*. The *car following* behavior describes the behavior of the vehicle in its longitudinal dynamics while the *lane change* behavior describes the behavior in the lateral position. 

In order to modify traffic behavior for a condition, the system is modeled via traffic model where V2I messages modify vehicle speed or lateral position   

#### Traffic model and the fundamental diagram

Before pursuing it is important to understand the value of the fundamental diagram. The fundamental diagram describe the relation ship between ***density*** or concentration of vehicles and the ***flow*** or speed of vehicles. The density is regularly denoted as $\kappa$ for this scenarios as `K`, in the meanwhile the flow is denoted by `Q`. We are going to consider the fact that the relationship follows a piecewise linear behavior as follows:


\begin{aligned}
Q(\kappa) = \begin{cases}
u\kappa\quad 0\leq\kappa\leq \kappa_c\\
w\left(\kappa_x-\kappa\right)\quad \kappa_c\leq\kappa\leq \kappa_x
\end{cases}
\end{aligned}


Let's define the number of vehicles and their initial positions, in order to determine the initial position let's find the minimum spacing. For the moment let's consider the following parameters. `W=5`, `K_X=0.2` and `U=20`. Let's trace the fundamental diagram for this case. We will make use of the `FundamentalDiagram` class in the `traffic` submodule

In [6]:
from vplatoon.traffic import FundamentalDiagram
from bokeh.plotting import show

f = FundamentalDiagram(w=5,u=25,k_x=0.2)
p = f.plot_diagram()
show(p)

We can easily establish the ***critical density*** which corresponds to the amount of vehicles at which there is maximum flow via

In [7]:
print(f"Capacity: {f.C}")
print(f"Critical density: {f.k_c}")
print(f"Maximum density: {f.k_x}")
print(f"Speed limit: {f.u}")
print(f"Congestion speed: {f.w}")

Capacity: 0.8333333333333334
Critical density: 0.03333333333333333
Maximum density: 0.2
Speed limit: 25
Congestion speed: 5


The critical density represent the minimum tolerable inter-vehicle distance **critical headway space** before vehicles start to decrease their speed. On the other hand when the value of density is maximum, and the flow is 0, the vehicles respect the **minimum headway space** that can be computed from the maximum value of density $\kappa_x$ via

\begin{aligned}
\kappa_x = \frac{N}{d} \approx \frac{1}{s_0}  \approx \frac{1}{l_{\text{avg}}+s_{\text{min}}} 
\end{aligned}

Where $l_{\text{avg}}$ is the average vehicle length and $s_{text{min}}$ represents the minimum tolerated headway spacing between two vehicles. 

By computing the case in the fundamental diagram

In [8]:
print(f"Minimum headway space: {1/f.k_x}")

Minimum headway space: 5.0


**Q1**: Supose a vehicle has an average length of $l_{\text{avg}}=4$ [m] and the minimum tolerable inter vehicle distance is $s_{\text{min}}=2.25$[m]. Consider the congestion wave speed $w=6.25$ [m/s] and a free flow speed of $u=25$ [m/s]. Compute and plot the fundamental diagram for this case and find the value of the capacity and minimum headway space:

<span style= "color:blue">Provide your answers below:</span>

#### Platooning behaviors and car following model 

Keep in mind the constants you have computed `K_X`, `W`, `U`, these parameters although apply for pure [car-following](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) they will be useful to construct and understand the behavior of truck platoons. 

For the sake of clarity let's consider the following figure.

<img src="assets/img/p2-cavs.png" alt="drawing" width="600"/>

For the sake of clarity, the following corresponds to the notations for variable description in the model. It is considered the vehicle position of a vehicle as $x_n$ and the headway space between a vehicle and its leader as $s_n = x_{n-1}-x_{n}$. The vehicle's speed and acceleration are defined as $v_n$,$a_n$ respectively. The operator $\Delta v_n = v_{n-1} - v_{n}$ refers to the difference between the leading vehicle and the following one. 

By introducing newton's law over one of the vehicles we can consider the following force set up and compute the accelertion given by the following expression: 

$$
a_n(t+T) = a_n(t) + T \frac{{u(t) - Mg\mu - {\scriptsize\frac{1}{2}}\rho C_d A v^2_n(t)}}{M} = a_n(t) + T g(C_d,v_n, u_n)
$$

Where $m$ corresponds to the truck mass, $C_d$ the drag coefficient, $A$ the frontal truck area, $\rho$ is the density of the air.  

Each one of the vehicles can follow the dynamics given by the following dynamic equation

\begin{aligned}
x_n(t+T) &= x_n(t) + v_n(t)T\\
v_n(t+T) &= v_n(t) + a_n(t) T\\
a_n(t+T) &= a_n(t) + T g(C_d,v_n,u_n)\\
\end{aligned}

*To implement the model a `class` object called `PlatoonDyn` has been implemented. The class intends to emulate these equations and keep track of the vehicle evolution. 

#### Parameters 

So far parameters in the model have been fixed although random scenarios can be also considered

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$M$$           | 0.5       |       |
$$A$$           | 10        | [m$^2$]
$$\rho$$        | 1.3       | [kg/m$^3$]
$${C_d}^\star$$ | 0.5 - 1.5 | 
$$\tau$$        | $$\frac{1}{wk_x}$$ | [s]
$$w$$           | $$6.25$$  | [m/s]
$$k_x$$         | $$0.16$$  | [veh/km]
$$u_i$$         | $$25$$    | [m/s]


$\star$: ${C_d}$ although is presented as parameter is a coefficient that changes according to the intervehicle distance. For the moment we have not considered the model but we will introduce this further in the system. 

Please follow step by step variable definition for more detail into simulations. Let define in this case the parameters `M`, `A`, `W`,`K_X` and import the class `PlatoonDyn` from the submodule `vplatoon.carfollow`

Creating the environment for this notebook 

In [10]:
from bokeh.plotting import figure
from vplatoon.carfollow import VehiclePlatoon
import numpy as np

In [12]:
d1 = VehiclePlatoon(0,25,0,"HDV")

u = np.concatenate([np.ones(10),np.ones(30)])

for ctr in u:
    d1(ctr)

In [13]:
p = figure()
p.line(np.arange(len(d1.a)),d1.a)


# len(g2.x)
show(p)

In [12]:
N_VEH = 2 # Number of vehicles 
X0 = np.array([25,0])
V0 = np.ones(N_VEH) * f.u
A0 = np.zeros(N_VEH)
V_CLASS = ["HDV", "HDV"]

veh_list = []
RegularVehicle.reset()
for x0, v0, vtype in zip(X0, V0, V_CLASS):
    veh_list.append(RegularVehicle (x0=x0, v0=v0, veh_type=vtype))
print(f"List of vehicles: {veh_list}")

TypeError: __init__() got an unexpected keyword argument 'x0'

In [2]:
from vplatoon.carfollow import OpenLoop

## Understanding traffic models, capacity

Initially, consider a set the fundamental diagram of a simple traffic model. The regular fundamental diagram is given by the following figure: 

![](img/fund-diag.png)

The parameters in this diagram represent the density of vehicles $\kappa$ [veh/m], and the average flow $Q(\kappa)$. The objective of this diagram is to describe how the concentration of vehicles and the space headway among them affects the flow of vehicles. 

In this case, the relationship between the $\kappa$ and the average spacing of vehicles is:
$$\frac{1}{s_0 + L_{\text{avg}}}=\kappa_x$$  
In this definition $s_0$ represents the minimum spacing allowed for vehicles and $L_{\text{avg}}$ represents the average length of a vehicle in the network. 

In the figure $u$ denotes the maximum speed limit in the highway and $w$ denotes the speed at which congestion is propagated. 

Let suppose a value of speed limit of *90* [Km/h] and a minimum spacing of 2[m]

**Q1:** <span style ="color:darkblue">
Design and write a python function that computes the flow in terms of the density and the parameters given as entries `fundamental_diagram(u,w,k)` find the maximum flow for $u=25$ [m/s]. Consider that $L_{\text{avg}}=4.25$ [m] and the minimum spacing $s_0 = 2$ [m]. Consider a value of $w=-6.25$[m/s]. Once created provide a plot of the function by using the libraries `numpy` and `matplotlib` 
</span>

**Q2:**: <span style ="color:darkblue">
Determine the maximum value of flow for this case and the corresponding density at which this value occurs
</span>    

## Setting up initial conditions for a platoon of two vehicles 

Imagine that vehicles are going to be split in an homogeneous way, 

**Q2:**

## Open loop for vehicle platooning 

**Q3:**

## Closin loop for vehicle platooning
**Q4:**

## Calibrating a consumption model 

**Q5:**

## Estimating emissions for a coupled model 

**Q6:**

## Estimating cumulated emissions 
**Q7:**

## Defining the 
**Q8**:

A. Ladino